In [1]:
#!pip install datasets

## Imports

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader, TensorDataset

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [66]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
import torch
import torch.nn.functional as F
import os
from collections import defaultdict

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/drive/My Drive/datasets

/content/drive/.shortcut-targets-by-id/1CNLowRdVtLOrHbCZbdjg3IVys53iPk7m/datasets


## Dataset

O dataset usado é o sobre notícias, para classificá-las conforme a categoria desta.

In [7]:
import csv
import pandas as pd

input_file = 'articles.csv'
output_file = 'corrected_articles.csv'

# Lista para armazenar as linhas válidas
data = []

# Ler o arquivo CSV manualmente
with open(input_file, 'r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    header = next(reader)  # Ler o cabeçalho
    for row in reader:
        data.append(row)

# Criar o DataFrame a partir dos dados
art_df = pd.DataFrame(data, columns=header)


In [8]:
art_df.columns
art_df = art_df.drop(columns = ['title','date','subcategory', 'link'])

 Conforme a documentação do site do Hugging Face, é necessário realizar a tokenização dos textos que serão utilizados no treinamento para o fine-tuning desses modelos. Dessa forma, e ate por uma questão de memoria, optei por manter apenas as colunas que contêm o corpo da notícia e a categoria à qual ela pertence.


In [9]:
from sklearn.preprocessing import LabelEncoder

# Codificar as categorias
label_encoder = LabelEncoder()
art_df['category'] = label_encoder.fit_transform(art_df['category'])
art_df = art_df.rename(columns={'category': 'label'})


As classes estão no formato de texto, o que não é ideal para o treinamento do modelo. Portanto, a função `label_encoder` converte as classes nominais em formatos numéricos

In [10]:
#Aplicação da operação usando pandas e tqdm para acompanhar o progresso
from tqdm import tqdm
tqdm.pandas()  # Ativar tqdm com pandas

# Função para truncar o texto após o último ponto
def truncate_text(s):
    if isinstance(s, str):
        return s[:s.rfind('.') + 1]
    return ''

# Aplicar a função para truncar o texto
art_df['text'] = art_df['text'].progress_apply(truncate_text)


100%|██████████| 167053/167053 [00:00<00:00, 425392.92it/s]


Durante o processo de tokenização, estava enfrentando problemas. A tokenização retornava erros e não funcionava, até que ao buscar tutoriais, foi possível perceber que o problema era que o texto não estava sendo encerrado com '.'. Ou seja, mesmo que o corpo da notícia tivesse terminado, havia ruídos como propaganda após o corpo da mensagem, o que necessitava de uma limpeza para manter o padrão. O Transformer interpreta ',' como uma forma de parada, então a falta de finalização da mensagem acabava causando erro. `trucate_text` resolve esse problema.

In [11]:
# Divida o DataFrame em treinamento (80%) e uma parte temporária (20%)
df_train, df_temp = train_test_split(art_df, test_size=0.9999, random_state=10)

# Divida a parte temporária em validação (50%) e teste (50%)
df_val, df_test = train_test_split(df_train, test_size=0.5, random_state=42)

Esta etapa de divisão dos dados é bastante delicada. O conjunto de dados é muito grande e estava estourando a memória, então precisei utilizar uma configuração que permitisse executar os experimentos. Como acredito que o objetivo da atividade seja testar nossas habilidades com os modelos, aprender a pré-processá-los, entre outros aspectos, não vejo problema quanto ao tamanho dos dados que consigo usar nos experimentos.

In [12]:
df_train.shape, df_test.shape

((16, 2), (8, 2))

In [13]:
import pandas as pd
import itertools

def remove_missing_classes(df_train, df_test):
  """
  Removes rows in df_train where the label is present in df_train but not in df_test.

  Args:
      df_train (pandas.DataFrame): The training DataFrame.
      df_test (pandas.DataFrame): The testing DataFrame.

  Returns:
      pandas.DataFrame: The filtered df_train with missing classes removed.
  """

  classes_train = set(df_train['label'].unique())
  classes_test = set(df_test['label'].unique())

  missing_classes = classes_train.difference(classes_test)

  return df_train[~df_train['label'].isin(missing_classes)]

# Example usage:
df_train = remove_missing_classes(df_train, df_test)
print(set(df_train['label']))
print(set(df_test['label'].unique()))


{35, 9, 12, 46, 26}
{35, 9, 12, 46, 26}


Devido ao tamanho do dataset, algumas classes estão presentes no conjunto de treino mas ausentes no conjunto de teste, o que gera problemas. Por isso, a função `removing_missing_classes` tem o objetivo de garantir que ambos os conjuntos tenham as mesmas classes.

## Função de Treino

A função `train_evaluate_model` foi criada para ser utilizada com os dois modelos solicitados, permitindo uma comparação mais justa dos resultados. Para isso, ela recebe os modelos, tokenizers, além dos conjuntos de treino (já tokenizados) e teste

In [71]:
def train_evaluate_model(tokenizer, model, tokenized_train, tokenized_test, y_train, y_test):

    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

    train_dataset = TensorDataset(
        tokenized_train["input_ids"].clone().detach(),
        tokenized_train["attention_mask"].clone().detach(),
        torch.tensor(y_train)
    )
    test_dataset = TensorDataset(
        tokenized_test["input_ids"].clone().detach(),
        tokenized_test["attention_mask"].clone().detach(),
        torch.tensor(y_test)
    )

    train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device='cpu'
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)

    for epoch in range(1):  # Loop over epochs
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch

            # Debugging: Print shapes and values
            #print("Batch input_ids shape train :", input_ids.shape)
            #print("Batch attention_mask shape train :", attention_mask.shape)
            #print("Batch labels shape train:", labels.shape)

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    model.eval()
    eval_loss = 0
    acc = []
    prec =[]
    num_classes = len(set(y_test))
    overall_cm = np.zeros((num_classes, num_classes), dtype=int)

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch

            # Debugging: Print shapes and values
            #print("Batch input_ids shape:", input_ids.shape)
            #print("Batch attention_mask shape:", attention_mask.shape)
            #print("Batch labels shape:", labels.shape)

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            predictions = torch.argmax(outputs.logits, dim=1)

            eval_loss += outputs.loss.item()



            accuracy = accuracy_score(labels, predictions)
            acc.append(accuracy)

            precision = precision_score(labels, predictions, average="weighted")
            prec.append(precision)

            print(f"Accuracy: {accuracy:.3f}")
            print(f"Precision: {precision:.3f}")


        average_accuracy = np.mean(acc)
        average_precision = np.mean(prec)

        print(f"Average Accuracy: {average_accuracy:.3f}")
        print(f"Average Precision: {average_precision:.3f}")
        print(f"Evaluation Loss: {eval_loss / len(test_dataloader)}")





## Carregamento dos Bertimbal

Vamos fazer tudo primeiro pra o Bertimbal e depois para o Bert

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Carregamento dos modelos BERT e Bertimbal

bertimbal_tokenizer = AutoTokenizer.from_pretrained("tiagoblima/newsdata-bertimbal")
bertimbal_model = AutoModelForSequenceClassification.from_pretrained("tiagoblima/newsdata-bertimbal")


## Peparação final dos dados

Os datasets precisam ser objetos do tipo dataset. Portanto, é necessário converter tanto o conjunto de treino quanto o de teste para esse formato. Além disso, como a seleção dos dados anteriormente foi feita de maneira aleatória, as classes resultantes no meu dataset podem não estar ordenadas. Por isso, a função convert_class_labels associa cada classe a uma ordem numérica (0, 1, 2, etc.), evitando problemas durante o treinamento.

In [17]:
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

In [18]:
y_train = dataset_train['label']
y_test = dataset_test['label']

In [19]:
def convert_class_labels(labels):
  """
  Converts a list of class labels (integers) to numerical representations (starting from 0).

  Args:
      labels (list): A list of integer class labels.

  Returns:
      torch.Tensor: A tensor containing the numerical representations of the class labels.
  """

  # Create a dictionary to map unique class labels to numerical representations
  class_to_index = {label: i for i, label in enumerate(sorted(set(labels)))}

  # Convert each label in the original list to its numerical representation
  numerical_labels = [class_to_index[label] for label in labels]

  # Convert the list to a PyTorch tensor
  return numerical_labels

In [20]:
y_train = convert_class_labels(y_train)
y_test =  convert_class_labels(y_test)

In [77]:
set(y_train), set(y_test), len(dataset_test["text"]), len(y_test), len(dataset_train["text"]), len(y_train)

({0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, 8, 8, 12, 12)

## Tokenização, Treino e Avaliação dos modelos.

In [21]:
tokenized_bertimbau_train = bertimbal_tokenizer(dataset_train["text"], padding="max_length", truncation=True, max_length=300, return_tensors="pt")

In [22]:
tokenized_bertimbau_test = bertimbal_tokenizer(dataset_test["text"], padding="max_length", truncation=True, max_length=300, return_tensors="pt")

In [72]:
train_evaluate_model(bertimbal_tokenizer, bertimbal_model, tokenized_bertimbau_train, tokenized_bertimbau_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Accuracy: 1.000
Precision: 1.000
Accuracy: 1.000
Precision: 1.000
Average Accuracy: 1.000
Average Precision: 1.000
Evaluation Loss: 0.0019725957536138594


O modelo é avaliado em batches, por isso imprimo as acurácias e precisões de cada um. Como ambos são 1, não conseguimos visualizar quais classes tiveram mais erros/acertos. Além disso, a perda (loss) é bastante pequena, o que faz sentido dado essas métricas.

## Fazendo o mesmo pra o Bert.

In [ ]:
def train_evaluate_model(tokenizer, model, tokenized_train, tokenized_test, y_train, y_test):

    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

    train_dataset = TensorDataset(
        tokenized_train["input_ids"].clone().detach(),
        tokenized_train["attention_mask"].clone().detach(),
        torch.tensor(y_train)
    )
    test_dataset = TensorDataset(
        tokenized_test["input_ids"].clone().detach(),
        tokenized_test["attention_mask"].clone().detach(),
        torch.tensor(y_test)
    )

    train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device='cpu'
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)

    for epoch in range(1):  # Loop over epochs
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch

            # Debugging: Print shapes and values
            #print("Batch input_ids shape train :", input_ids.shape)
            #print("Batch attention_mask shape train :", attention_mask.shape)
            #print("Batch labels shape train:", labels.shape)

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    model.eval()
    eval_loss = 0
    acc = []
    prec =[]
    num_classes = len(set(y_test))
    overall_cm = np.zeros((num_classes, num_classes), dtype=int)
    guessed_classes = []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch

            # Debugging: Print shapes and values
            #print("Batch input_ids shape:", input_ids.shape)
            #print("Batch attention_mask shape:", attention_mask.shape)
            #print("Batch labels shape:", labels.shape)

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            predictions = torch.argmax(outputs.logits, dim=1)
            for i in range(len(labels)):
                 if predictions[i] != labels[i]:
                    guessed_classes.append((predictions[i].item(), labels[i].item()))

            eval_loss += outputs.loss.item()



            accuracy = accuracy_score(labels, predictions)
            acc.append(accuracy)

            precision = precision_score(labels, predictions, average="weighted")
            prec.append(precision)

            print(f"Accuracy: {accuracy:.3f}")
            print(f"Precision: {precision:.3f}")


        average_accuracy = np.mean(acc)
        average_precision = np.mean(prec)

        print(f"Average Accuracy: {average_accuracy:.3f}")
        print(f"Average Precision: {average_precision:.3f}")
        print(f"Evaluation Loss: {eval_loss / len(test_dataloader)}")


        for guessed, correct in guessed_classes:
            print(f"Guessed: {guessed} (Correct: {correct})")




In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Carregamento dos modelos BERT e Bertimbal

bert_tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
bert_model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=len(set(y_train)))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_bert_train = bert_tokenizer(dataset_train["text"], padding="max_length", truncation=True, max_length=300, return_tensors="pt")

In [ ]:
tokenized_bert_test = bert_tokenizer(dataset_test["text"], padding="max_length", truncation=True, max_length=300, return_tensors="pt")

O modelo é avaliado em batches, por isso imprimo as acurácias e precisões de cada um. Como ambos são 1, não conseguimos visualizar quais classes tiveram mais erros/acertos. Além disso, a perda (loss) é bastante pequena, o que faz sentido dado essas métricas.

In [ ]:
train_evaluate_model(bert_tokenizer, bert_model, tokenized_bert_train, tokenized_bert_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.400
Precision: 0.400
Accuracy: 0.667
Precision: 0.500
Average Accuracy: 0.533
Average Precision: 0.450
Evaluation Loss: 1.2417539358139038
Guessed: 1 (Correct: 4)
Guessed: 1 (Correct: 0)
Guessed: 1 (Correct: 2)
Guessed: 1 (Correct: 0)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_train

[3, 4, 1, 3, 1, 3, 1, 1, 0, 2, 2, 0]

Como o BERT apresentou erros, criei uma função modificada para ele, que nos permite saber qual classe ele previu e qual era a correta. Podemos observar que ele geralmente prevê a classe 1, que é a classe com o maior número de amostras.

 ## Conclusao

O BERTimbau teve a melhor performance, o que era esperado, considerando que ele já foi pré-treinado em português, ao contrário do BERT. Além disso, o BERT tende a prever a classe com maior disponibilidade no dataset.